# Manufacturing Challenge 2 - Baseline Submission

This notebook provides a simple baseline for **Manufacturing Challenge 2: Quality Cost Prediction**.

**Goal**: Predict `ReworkCostUSD` for each production unit
**Metric**: Normalized Gini - Higher is better

## Instructions:
1. **Replace API credentials** in the first cell with your team's API key and name
2. **Run all cells** to generate and submit baseline predictions
3. **Check the output** for your submission score

This baseline uses only tabular batch production data with a simple Random Forest regressor.


In [ ]:
# 1. Initialize Client and Load Data

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from agentds import BenchmarkClient

# 🔑 REPLACE WITH YOUR CREDENTIALS
client = BenchmarkClient(
    api_key="your-api-key-here",        # Get from your team dashboard
    team_name="your-team-name-here"     # Your exact team name
)

# Load data from PVC paths
print("📂 Loading Manufacturing Challenge 2 data...")

# Load batch production data
train_batches = pd.read_csv("/home/jovyan/shared/datasets/Manufacturing/batches_train.csv")
test_batches = pd.read_csv("/home/jovyan/shared/datasets/Manufacturing/batches_test.csv")

print(f"✅ Data loaded:")
print(f"   Train batches: {train_batches.shape}")
print(f"   Test batches: {test_batches.shape}")
print(f"   Train columns: {list(train_batches.columns)}")
print(f"   Test columns: {list(test_batches.columns)}")


In [ ]:
# 2. Tabular-Only Baseline Model and Predictions

# From data inspection - batches columns:
# unit_id, machine_id, speed_rpm, feed_mm_rev, coolant_pct, target_temp_c, ReworkCostUSD (train only)

# Select numeric production features for baseline
production_features = ['speed_rpm', 'feed_mm_rev', 'coolant_pct', 'target_temp_c']
print(f"📊 Using production features: {production_features}")

# Prepare training data
X_train = train_batches[production_features].fillna(0)
y_train = train_batches['ReworkCostUSD']  # Target variable

# Prepare test data
X_test = test_batches[production_features].fillna(0)

# Train simple Random Forest baseline
print("🤖 Training Random Forest regressor...")
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Create submission file (format: unit_id,ReworkCostUSD)
submission_df = pd.DataFrame({
    'unit_id': test_batches['unit_id'],
    'ReworkCostUSD': predictions
})

# Save predictions
submission_df.to_csv("manufacturing_challenge2_predictions.csv", index=False)
print(f"✅ Predictions saved: {submission_df.shape[0]} predictions")
print(f"   Preview: {submission_df.head(3)}")
print(f"   Cost range: ${predictions.min():.2f} to ${predictions.max():.2f}")


In [ ]:
# 3. Submit Predictions

# Submit predictions to the competition
print("🚀 Submitting predictions...")

try:
    result = client.submit_prediction("Manufacturing", 2, "manufacturing_challenge2_predictions.csv")
    
    if result['success']:
        print("✅ Submission successful!")
        print(f"   📊 Score: {result['score']:.4f}")
        print(f"   📏 Metric: {result['metric_name']}")
        print(f"   ✔️  Validation: {'Passed' if result['validation_passed'] else 'Failed'}")
    else:
        print("❌ Submission failed!")
        print(f"   Error details: {result.get('details', {}).get('validation_errors', 'Unknown error')}")
        
except Exception as e:
    print(f"💥 Submission error: {e}")
    print("🔧 Check your API key and team name are correct!")

print("\n🎯 Next steps:")
print("   1. Try incorporating relevant information outside this table!")
print("   2. Move on to Manufacturing Challenge 3!")
